# Customer LifeTime Value (CLV)

### Load Data 

In [6]:
import pandas as pd
data=pd.read_excel("/content/Online Retail.xlsx")
data.head() 

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Data Information 

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
data.isnull().sum() # mising value 

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

### Data Pre-processing

In [9]:
newdf = data.drop_duplicates() # removing duplicates 

In [10]:
newdf.info() # new data,  after remove the duplicates 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 536641 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    536641 non-null  object        
 1   StockCode    536641 non-null  object        
 2   Description  535187 non-null  object        
 3   Quantity     536641 non-null  int64         
 4   InvoiceDate  536641 non-null  datetime64[ns]
 5   UnitPrice    536641 non-null  float64       
 6   CustomerID   401604 non-null  float64       
 7   Country      536641 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 36.8+ MB


In [11]:
# removing null values 
newdf = newdf.dropna()

In [12]:
newdf.isnull().any() # checking wheather null is there or not 

InvoiceNo      False
StockCode      False
Description    False
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID     False
Country        False
dtype: bool

In [13]:
newdf.describe() #  statistical approch of numeric data

,Quantity,UnitPrice,CustomerID
count,401604.000000,401604.000000,401604.000000
mean,12.183273,3.474064,15281.160818
std,250.283037,69.764035,1714.006089
min,-80995.000000,0.000000,12346.000000
25%,2.000000,1.250000,13939.000000
50%,5.000000,1.950000,15145.000000
75%,12.000000,3.750000,16784.000000
max,80995.000000,38970.000000,18287.000000


In [14]:
# Here, you can observe some of the customers have ordered in a negative quantity,
#  which is not possible.  So, need to filter Quantity greater than zero. 

In [15]:
# filter the quantity data
data = newdf[(newdf['Quantity']>0)]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392732 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    392732 non-null  object        
 1   StockCode    392732 non-null  object        
 2   Description  392732 non-null  object        
 3   Quantity     392732 non-null  int64         
 4   InvoiceDate  392732 non-null  datetime64[ns]
 5   UnitPrice    392732 non-null  float64       
 6   CustomerID   392732 non-null  float64       
 7   Country      392732 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 27.0+ MB


###  Filter Features for calculate CLV

Here, you can filter the necessary columns for calculating CLTV. You only need her five columns CustomerID, InvoiceDate, InvoiceNo, Quantity, and UnitPrice.

CustomerID will uniquely define your customers.


InvoiceDate help you calculate numbers of days customer stayed with your product.

InvoiceNo helps you to count the number of time transaction performed(frequency)

Quantity is purchased item units in each transaction

UnitPrice of each unit purchased by the customer will help you to calculate the total purchased amount.

In [16]:
fdata=data[['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]

In [17]:
fdata.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice
0,17850.0,2010-12-01 08:26:00,536365,6,2.55
1,17850.0,2010-12-01 08:26:00,536365,6,3.39
2,17850.0,2010-12-01 08:26:00,536365,8,2.75
3,17850.0,2010-12-01 08:26:00,536365,6,3.39
4,17850.0,2010-12-01 08:26:00,536365,6,3.39


In [18]:
#Calulate total price - using unit price and quantity 
fdata['TotalPrice'] = fdata['Quantity'] * fdata['UnitPrice']

<ipython-input-18-ae95ab668d99>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdata['TotalPrice'] = fdata['Quantity'] * fdata['UnitPrice']


In [19]:
fdata.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice
0,17850.0,2010-12-01 08:26:00,536365,6,2.55,15.30
1,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34
2,17850.0,2010-12-01 08:26:00,536365,8,2.75,22.00
3,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34
4,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34


Calculate following to find clv 

Calculate the number of days between the present date and the date of last purchase from each customer.

Calculate the number of orders for each customer.

Calculate sum of purchase price for each customer.

In [20]:
data_group=fdata.groupby('CustomerID').agg({'InvoiceDate': lambda date: (date.max() - date.min()).days,
                                        'InvoiceNo': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'TotalPrice': lambda price: price.sum()})

In [21]:
data_group.head()

,InvoiceDate,InvoiceNo,Quantity,TotalPrice
CustomerID,,,,
12346.0,0,1,74215,77183.60
12347.0,365,182,2458,4310.00
12348.0,282,31,2341,1797.24
12349.0,0,73,631,1757.55
12350.0,0,17,197,334.40


In [22]:
# Change the name of columns
data_group.columns=['no_days','no_transactions','no_units','spent_money']
data_group.head()

,no_days,no_transactions,no_units,spent_money
CustomerID,,,,
12346.0,0,1,74215,77183.60
12347.0,365,182,2458,4310.00
12348.0,282,31,2341,1797.24
12349.0,0,73,631,1757.55
12350.0,0,17,197,334.40


# Calculate LifeTime Value (CLV)

In [23]:
# The following formulae is used for calculating customer lifetime value 

# CLV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.

#  Customer Value = Average Order Value * Purchase Frequency

###  Calculate Average Order Value

In [24]:
# Average Order Value
data_group['avg_order_value']=data_group['spent_money']/data_group['no_transactions']

In [25]:
data_group.head()

,no_days,no_transactions,no_units,spent_money,avg_order_value
CustomerID,,,,,
12346.0,0,1,74215,77183.60,77183.600000
12347.0,365,182,2458,4310.00,23.681319
12348.0,282,31,2341,1797.24,57.975484
12349.0,0,73,631,1757.55,24.076027
12350.0,0,17,197,334.40,19.670588


### Calculate Purchase Frequency

In [26]:
#number of customer divided by  total number of customer 

In [27]:
purchase_frequency=sum(data_group['no_transactions'])/data_group.shape[0]

In [28]:
purchase_frequency

90.51209956211109

### Calculate Repeat Rate and Churn Rate

In [29]:
# Repeat Rate
repeat_rate=data_group[data_group.no_transactions > 1].shape[0]/data_group.shape[0]

In [30]:
repeat_rate

0.9834063148190827

In [31]:
#Churn Rate
churn_rate=1-repeat_rate

In [32]:
churn_rate

0.016593685180917306

### Calculate Profit Margin

Profit margin is the commonly used profitability ratio. It represents how much percentage of total sales has earned as the gain. 

In [33]:
# Profit Margin
data_group['profit_margin']=data_group['spent_money']*0.05 # 0.05 is assume value, because we dont know the margin

In [34]:
data_group.head()

,no_days,no_transactions,no_units,spent_money,avg_order_value,profit_margin
CustomerID,,,,,,
12346.0,0,1,74215,77183.60,77183.600000,3859.1800
12347.0,365,182,2458,4310.00,23.681319,215.5000
12348.0,282,31,2341,1797.24,57.975484,89.8620
12349.0,0,73,631,1757.55,24.076027,87.8775
12350.0,0,17,197,334.40,19.670588,16.7200


### Calcualte Customer Lifetime Value

In [35]:
# Customer Value
data_group['CLV']=(data_group['avg_order_value']*purchase_frequency)/churn_rate

In [36]:
#Customer Lifetime Value
data_group['cust_lifetime_value']=data_group['CLV']*data_group['profit_margin']

In [37]:
data_group.head()

,no_days,no_transactions,no_units,spent_money,avg_order_value,profit_margin,CLV,cust_lifetime_value
CustomerID,,,,,,,,
12346.0,0,1,74215,77183.60,77183.600000,3859.1800,4.210065e+08,1.624740e+12
12347.0,365,182,2458,4310.00,23.681319,215.5000,1.291724e+05,2.783665e+07
12348.0,282,31,2341,1797.24,57.975484,89.8620,3.162337e+05,2.841739e+07
12349.0,0,73,631,1757.55,24.076027,87.8775,1.313254e+05,1.154054e+07
12350.0,0,17,197,334.40,19.670588,16.7200,1.072954e+05,1.793979e+06


# Predict CLTV using Linear Regression 

In [38]:
fdata.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice
0,17850.0,2010-12-01 08:26:00,536365,6,2.55,15.30
1,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34
2,17850.0,2010-12-01 08:26:00,536365,8,2.75,22.00
3,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34
4,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34


In [39]:
#fdata['month_year'] = fdata['InvoiceDate'].dt.to_period('M') # extract month and year 

<ipython-input-39-73fbea6d9c13>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fdata['month_year'] = fdata['InvoiceDate'].dt.to_period('M')


In [67]:
fdata['month_yr'] = fdata['InvoiceDate'].apply(lambda x: x.strftime('%b-%Y'))

In [68]:
fdata.head()

,CustomerID,InvoiceDate,InvoiceNo,Quantity,UnitPrice,TotalPrice,month_year,month_yr
0,17850.0,2010-12-01 08:26:00,536365,6,2.55,15.30,2010-12,Dec-2010
1,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34,2010-12,Dec-2010
2,17850.0,2010-12-01 08:26:00,536365,8,2.75,22.00,2010-12,Dec-2010
3,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34,2010-12,Dec-2010
4,17850.0,2010-12-01 08:26:00,536365,6,3.39,20.34,2010-12,Dec-2010


In [69]:
fdata["month_yr"].unique()

array(['Dec-2010', 'Jan-2011', 'Feb-2011', 'Mar-2011', 'Apr-2011',
       'May-2011', 'Jun-2011', 'Jul-2011', 'Aug-2011', 'Sep-2011',
       'Oct-2011', 'Nov-2011', 'Dec-2011'], dtype=object)

The pivot table takes the columns as input, and groups the entries into a two-dimensional table in such a way that provides a multidimensional summarization of the data.

In [70]:
sales=fdata.pivot_table(index=['CustomerID'],columns=['month_yr'],values='TotalPrice',aggfunc='sum',fill_value=0).reset_index()

In [71]:
sales.head()

month_yr,CustomerID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011
0,12346.0,0.00,0.00,0.00,0.00,0.0,77183.60,0.0,0.00,0.0,0.0,0.00,0.00,0.0
1,12347.0,636.25,584.91,711.79,224.82,0.0,475.39,0.0,382.52,0.0,0.0,0.00,1294.32,0.0
2,12348.0,367.00,0.00,892.80,0.00,0.0,227.44,0.0,0.00,0.0,0.0,0.00,0.00,310.0
3,12349.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,1757.55,0.00,0.0
4,12350.0,0.00,0.00,0.00,0.00,334.4,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0


In [72]:
# Let's sum all the months sales.
sales['CLV']=sales.iloc[:,2:].sum(axis=1)

In [73]:
sales.head()

month_yr,CustomerID,Apr-2011,Aug-2011,Dec-2010,Dec-2011,Feb-2011,Jan-2011,Jul-2011,Jun-2011,Mar-2011,May-2011,Nov-2011,Oct-2011,Sep-2011,CLV
0,12346.0,0.00,0.00,0.00,0.00,0.0,77183.60,0.0,0.00,0.0,0.0,0.00,0.00,0.0,77183.60
1,12347.0,636.25,584.91,711.79,224.82,0.0,475.39,0.0,382.52,0.0,0.0,0.00,1294.32,0.0,3673.75
2,12348.0,367.00,0.00,892.80,0.00,0.0,227.44,0.0,0.00,0.0,0.0,0.00,0.00,310.0,1430.24
3,12349.0,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,1757.55,0.00,0.0,1757.55
4,12350.0,0.00,0.00,0.00,0.00,334.4,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,334.40


In [74]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4339 entries, 0 to 4338
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   CustomerID  4339 non-null   float64
 1   Apr-2011    4339 non-null   float64
 2   Aug-2011    4339 non-null   float64
 3   Dec-2010    4339 non-null   float64
 4   Dec-2011    4339 non-null   float64
 5   Feb-2011    4339 non-null   float64
 6   Jan-2011    4339 non-null   float64
 7   Jul-2011    4339 non-null   float64
 8   Jun-2011    4339 non-null   float64
 9   Mar-2011    4339 non-null   float64
 10  May-2011    4339 non-null   float64
 11  Nov-2011    4339 non-null   float64
 12  Oct-2011    4339 non-null   float64
 13  Sep-2011    4339 non-null   float64
 14  CLV         4339 non-null   float64
dtypes: float64(15)
memory usage: 508.6 KB


In [75]:
sales.columns

Index(['CustomerID', 'Apr-2011', 'Aug-2011', 'Dec-2010', 'Dec-2011',
       'Feb-2011', 'Jan-2011', 'Jul-2011', 'Jun-2011', 'Mar-2011', 'May-2011',
       'Nov-2011', 'Oct-2011', 'Sep-2011', 'CLV'],
      dtype='object', name='month_yr')

In [76]:
X=sales[['Dec-2011','Nov-2011', 'Oct-2011','Sep-2011','Aug-2011','Jul-2011']]# imagine if we have only six month of data 
y=sales[['CLV']]

###  Splitting Data

In [80]:
#split training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

### Linear Regression

In [81]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred = linreg.predict(X_test)

In [82]:
# print the intercept and coefficients
print(linreg.intercept_)
print(linreg.coef_)

[70.45618465]
[[1.00813952 1.0777134  1.92632911 1.33275221 2.82694179 2.05933629]]


### Model Evaluation

In [83]:
from sklearn import metrics

# compute the R Square for model
print("R-Square:",metrics.r2_score(y_test, y_pred))

R-Square: 0.7324809573248418


In [85]:
# calculate MAE using scikit-learn
import numpy as np
print("MAE:",metrics.mean_absolute_error(y_test,y_pred))

#calculate mean squared error
print("MSE",metrics.mean_squared_error(y_test, y_pred))
# compute the RMSE of our predictions
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MAE: 682.826508777692
MSE 5859263.624523391
RMSE: 2420.591585650787
